# Buckling of a cylindrical shell

In the present notebook, the critical load of a uniformly compressed cylindrical shell is found.

A cylindrical shell is considered ($L$: length; $R$: radius; $h$: thickness).

The constitutive material follows a Kirchhoff--Saint-Venant law ($E$: Young modulus, $\nu$: Poisson ratio).

The shell is subjected to an axial compression $q$, so that the total applied force is $2\pi Rq$. For each value of the applied load $q$, we find a particular equilibrium $\mathbf u^\star(q)$, the so-called *fundamental path*. In the present case, the fundamental path is axisymmetric, constant in $z$. Note that $\mathbf u^\star(q)$ is in fact a field, that depends on the observation point $\mathsf M$: $\mathbf u^\star(\mathsf M, q)$.

We seek the *critical* value of $q$, $q_\mathrm{cr}$, for which the equilibrium on the fondamental path, $\mathbf u^\star(q_\text{cr})$ becomes unstable. The approach that we follow here is quite general: we first compute the total potential energy $\Pi(\mathbf u, q)$ of the system, *including geometric non-linearities*. For a fixed value of the loading parameter $q$, we then evaluate the total potential energy *in the neighborhood* of the fundamental path, and check whether the potentatial energy is indeed minimal. In other words, we must assess the sign of the following quantity

\begin{equation*}
\Pi[\mathbf u^\star(q)+\delta\mathbf u, q]-\Pi[\mathbf u^\star(q), q].
\end{equation*}

For a *fully rigorous stability analysis*, the above quantity should be evaluated for *any* test-function $\delta\mathbf u$. As a first step, we will evaluate the potential energy for a two-parameter family of test-functions. This therefore provides an estimate *in excess* of the critical load.

Note that the present derivation is somewhat tedious; we will therefore call the [sympy](https://docs.sympy.org/latest/index.html) library to the rescue.

The notebook is organized as follows. In Sec. 1, we define the geometry of the underformed cylinder.

## 0. Setting-up the stage

We first load the `sympy` module and define a few symbols to represent the parameters of the problem at hand.

In [ ]:
import sympy

Classical, letter-like symbols are defined through a simple import.

In [ ]:
from sympy.abc import h, q, z, D, R

For more complex definitions, you need to use the `Symbol` class, for example if you want to specify the sign of a constant (this will be used by sympy below to eliminate some discussions below)

In [ ]:
L = sympy.Symbol("L", positive=True)

or if you want to have better control over the way the symbols are displayed

In [ ]:
θ, Eh, ν = sympy.symbols(r"\theta Eh \nu")

**Notes:**
1. You can use the `symbols` function to define several symbols at a time. Their names should be separated by spaces or commas in the provided strings.
2. Note the prefix `r` in the above string. This defines a *raw* string, for which backslash is interpreted as is. In a regular string, remember that backslashes must be doubled like so `"\\theta Eh \\nu"` (overwise, `\t` is interpreted as tabulation, and `\n` as newline).
3. The bending stiffness of the cylinder is `D`, while its membrane stiffness is `Eh / (1 - ν**2)`, where `Eh` is a unique symbol. Of course, `D` and `Eh` are not independent, since `D = Eh * h** 2 / 12 / (1 - ν**2)`. Still, it will be convenient to use *two* different variables.

## 1. Definition of the geometry of the cylinder

The dimensions `R`, `L` , `h` have already been defined, as well as the curvilinear coordinates `θ` and `z`.

In what follows, vectors of the euclidean space will be expressed as column-vectors, while 2nd order tensors will be expressed as matrices. **We will systematically work in the cylindrical basis**, defined as follows

In [ ]:
e_r = sympy.Matrix([1, 0, 0])
e_θ = sympy.Matrix([0, 1, 0])
e_z = sympy.Matrix([0, 0, 1])

Note that the first component of a vector is therefore *normal* to the cylinder; in other words, the first component of a tangent vector must vanish. The projection onto the tangent plane has the following simple expression

In [ ]:
A = sympy.eye(3)
A[0, 0] = 0
A

Note that we chose to represent a second order tensor of the tangent plane as a 3×3, even if the first row and first column vanish. We now define the curvature tensor, $\mathbf B$ (beware the sign of the curvature!)

In [ ]:
B = sympy.zeros(3, 3)
B[1, 1] = -1 / R
B

Finally, we need to be able to compute the surface gradient of a vector field. We have the following formula

\begin{equation*}
\operatorname{\textbf{Grad}}\mathbf T=\frac1R \partial_\theta\mathbf T \otimes \mathbf{e}_\theta + \partial_z\mathbf T \otimes \mathbf{e}_z,
\end{equation*}

for any tensor field $\mathbf T$. For an axisymmetric vector field $\mathbf v=v_r(z)\,\mathbf{e}_r+v_\theta(z)\,\mathbf{e}_\theta+v_z(z)\,\mathbf{e}_z$, the above formula reads

\begin{equation*}
\begin{aligned}
\operatorname{\textbf{Grad}}\mathbf v &= \frac 1R\bigl(v_r\,\mathbf{e}_\theta - v_\theta\,\mathbf{e}_r\bigr)\otimes\mathbf{e}_r + \bigl(v_r'\,\mathbf{e}_r + v_\theta'\,\mathbf{e}_\theta + v_z' \,\mathbf{e}_z\bigr)\otimes\mathbf{e}_z\\
&= \frac 1R\bigl(\mathbf{e}_z\times\mathbf{v}\bigr) \otimes \mathbf{e}_\theta + v_i' \mathbf{e}_i\otimes \mathbf{e}_z,
\end{aligned}
\end{equation*}
where $'$ denotes derivative with respect to $z$.

In matrix form, remember that the tensor product of two vectors $\mathbf{v}$ and $\mathbf{w}$ is the following matrix product of their matrix representations $\mathsf{v}$ and $\mathsf{w}$: $\mathbf{v}\otimes\mathbf{w}\leftrightarrow\mathsf{v}\cdot\mathsf{w}^\mathsf{T}$. These considerations result in the following definition of the `Grad` function

In [ ]:
def Grad(v):
    return (e_z.cross(v) @ e_θ.T)/ R + v.diff(z) @ e_z.T

where `@` denotes the matrix-matrix product in Python.

## 2. Evaluation of the *exact* generalized strains

We need to evaluate the potential energy of the cylinder for any displacement field $\vec u$. We will restrict the analysis to axisymmetric modes. Note that for the purpose of stability analyses, this is generally not sufficient, as instability tends to **break symmetries**.

The displacement field we consider is therefore decomposed as follows

\begin{equation*}
\mathbf{u}=v(z)\,\mathbf{e}_z + w(z)\,\mathbf{e}_r,
\end{equation*}

where $v$ and $w$ are two functions of the variable $z$, defined as the following symbolic functions in `sympy`

In [ ]:
v = sympy.Function("v")
w = sympy.Function("w")

Note that the variables the these functions depend upon has not been specified yet. We just need to write `v(z)` to express that the function $v$ is evaluated at the point $z$. This means that `sympy` does *not* know the number of variables upon which these functions depend!

As already mentioned before, we need to account for *geometric non-linearities* in the present analysis. This means that the generalized strains will not be linearized with respect to $v$ and $w$. However, keeping all non-linear terms would lead to an overly complicated calculation. We will therefore keep only the minimal number of non linear terms.

It is generally assumed that the transverse displacement $w$ is of order 1, while the tangential displacement $v$ is of order 2. In order to facilitate the analysis of the order of the various terms, we introduce a fictitious, dimensionless gage parameter $\gamma$ as follows.

In [ ]:
γ = sympy.Symbol("\gamma", positive=True)
u = sympy.Matrix([γ * w(z), 0, γ**2 * v(z)])
u

The above definition indeed expresses that the $z$ component of the displacement is a second order term $\mathcal O(\gamma^2)$, while the normal component is of the first order $\mathcal O(\gamma)$. Note that in the `sympy` terminology, `u` is an expression, not a function: it is the displacement evaluated at `z`. Conversely, `v` and `w` are functions, and the evaluation point must be specified explicitly: `v(z)`, `w(z)`.

We are now ready to evaluate the generalized strains $\mathbf E$ and $\mathbf K$ induced by the above displacement. We start with the deformation gradient, $\mathbf F$

In [ ]:
F = A + Grad(u)
F

We then compute and expand the *exact* membrane strains

In [ ]:
E_exact = sympy.expand((F.T @ F - A) / 2)
E_exact

We now turn to the curvature change, $\mathbf{K}$, defined as follows

\begin{equation*}
\mathbf{K}
= \mathbf{F}^\mathsf{T}\cdot\mathbf{b}\cdot\mathbf{F}
= -\mathbf{F}^\mathsf{T}\cdot\operatorname{\textbf{grad}}\mathbf{n}\cdot\mathbf{F},
\end{equation*}

where $\mathbf{b}=-\operatorname{\textbf{grad}}\mathbf{n}$ denotes the *current* curvature tensor, and $\textbf{grad}$ is the surface gradient over the *current* configuration. We have shown the following formula that relates gradients on the initial and current configurations

\begin{equation*}
\operatorname{\textbf{grad}}\mathbf{n}\cdot\mathbf{F}=\operatorname{\textbf{Grad}}\mathbf{n},
\end{equation*}

that will be used below. We first compute the outer unit normal $\mathbf{n}$ to the *current* configuration as follows. The current point $\mathsf m$ on the current configuration is defined as follows

\begin{equation*}
\vec{\mathsf{m}} = \bigl(R + w\bigr)\mathbf{e}_r + \bigl(z + v\bigr)\mathbf{e}_z,
\end{equation*}

we therefore find the following tangent vectors

\begin{align*}
\partial_\theta\vec{\mathsf{m}} &= R\,\mathbf{e}_\theta + \mathbf{e}_z \times \vec u,\\
\partial_z\vec{\mathsf{m}} &= e_z + u_i'\mathbf{e}_i.
\end{align*}

In [ ]:
a_θ = R * e_θ + e_z.cross(u)
a_z = e_z + u.diff(z)

We then define the normal $\mathbf{n}$ as the normalized vector $\partial_\theta\vec{\mathsf m}\times\partial_z\vec{\mathsf m}$.

In [ ]:
n_ = sympy.factor(a_θ.cross(a_z) / (R + γ * w(z)))
n = n_ / n_.norm()

From which we find the exact expression of the curvature change.

In [ ]:
bF = -Grad(n)
K_exact = F.T @ bF - B
K_exact = K_exact.applyfunc(sympy.ratsimp)
K_exact

## Evaluation of the approximate generalized strains

Donnell's approximation of the generalized strains, that will be introduced below, corresponds to keeping the leading order term of each coefficient of $\mathbf{E}$ and $\mathbf{K}$. We use the method `leadterm`.

In [ ]:
E = E_exact.applyfunc(lambda expr: expr.leadterm(γ)[0])
E

It is observed that $E_{\theta\theta}$ is of order 1, while $E_{zz}$ is of order 2. We apply the same function to the coefficient of the curvature change tensor.

In [ ]:
K = K_exact.applyfunc(lambda expr: expr.leadterm(γ)[0])
K

We perform a further (not really justified!) simplification by nullifying the $K_{\theta\theta}$ coefficient.

In [ ]:
K[1, 1] = 0
K

We can now evaluate the potential energy of the cylindrical shell.

## Evaluation of the potential energy of the shell

We use the formula: $\Pi = \mathcal U - \mathcal V$, where $U$ is the strain energy of the shell and $V$ is the potential of external forces. The general expression of the strain energy is given by the following integral

\begin{equation*}
\mathcal U=\frac12\int_\Sigma\bigl(\mathbf{N}:\mathbf{E}+\mathbf{M}:\mathbf{K}\bigr)\,\mathrm{d}A,
\end{equation*}
where $\mathbf{N}$ and $\mathbf{M}$ are the generalized strains associated to $\mathbf{E}$ and $\mathbf{K}$. We first compute these generalized stresses.

In [ ]:
N = Eh / (1 - ν**2) * ((1 - ν) * E + ν * E.trace() * A)
M = D * ((1 - ν) * K + ν * K.trace() * A)

From which we deduce the expression of the strain energy

In [ ]:
𝒰 = 2 * sympy.pi * R * sympy.Integral(sympy.trace(N @ E + M @ K) / 2, (z, 0, L))

Note that we defined this expression as an `Integral` object, which represents an unevaluated integral.

The potential of external forces $\mathcal V$, is given by the following expression
\begin{equation*}
\mathcal V = -\int_{z=L} q\,v(L)\,\mathrm{d} s=-2\pi\,R\,q\,v(L).
\end{equation*}

In [ ]:
𝒱 = -2 * sympy.pi * R * q * v(L)

And we finally have the expression of the total potential energy $\Pi$.

In [ ]:
Π = 𝒰 - 𝒱
Π.expand()

## Equilibrium equations

The equilibrium equations express the *stationarity* of the energy. Therefore, we evaluate $\mathrm{D}\Pi$, the differential of the potential energy. We use the following practical definition (that requires sufficient regularity of $\Pi$) based on *directional derivatives*

\begin{equation*}
\mathrm{D}\Pi(v, w, q; \delta v, \delta w)=\lim_{t\to0}\frac{\Pi(v+t\,\delta v, w+t\,\delta w, q)-\Pi(v, w, q)}{t}
=\frac{\mathrm{d}}{\mathrm{d}t}\Pi(v+t\,\delta v, w+t\,\delta w, q)\bigl|_{t=0}.
\end{equation*}

We first define the test functions $\delta v$ and $\delta w$ and the parameter t.

In [ ]:
t = sympy.Symbol("t")
δv = sympy.Function(r"\delta v")
δw = sympy.Function(r"\delta w")

We then substitute $v+t\,\delta v\to v$ and $w+t\,\delta w\to w$ into the expression of the potential energy to obtain the perturbed potential energy, $\Pi'$.

In [ ]:
perturbation = {v(L): v(L) + t * δv(L), # TODO This repetition is uggly. But `v : v + δv` raises an exception
                v(z): v(z) + t * δv(z),
                w(z): w(z) + t * δw(z)}
Π_1 = Π.subs(perturbation)

In [ ]:
DΠ = Π_1.diff(t).subs(t, 0)

In [ ]:
DΠ.expand()

## Fundamental branch

We seek a particular solution of the form $v = \alpha\, z$ and $w = \beta\, R$, where $\alpha$ and $\beta$ are two constants. To find the value of $\alpha$ and $\beta$, we first minimize the energy with respect to to these two scalars. We will then need to check that the residual $\mathrm{D}\Pi$ vanishes for *all* test functions.

We first define the unknowns $\alpha$ and $\beta$.

In [ ]:
α, β = sympy.symbols(r"\alpha \beta")

We then substitute the expressions of $v^\star$ and $w^\star$ in the potential energy, which becomes a function of $\alpha$, $\beta$ and $q$ only.

In [ ]:
fundamental = {v(x): α * x for x in [0, L, z]}
fundamental.update({w(x): β * R for x in [0, L, z]})
fundamental

The total potential energy now becomes a function of $\alpha$ an $\beta$ only.

In [ ]:
Π_star = Π.subs(fundamental).doit().factor()
Π_star

To find the optimal values of $\alpha$ and $\beta$, we write that both derivatives with respect to these two variables must vanish. This leads to the following equations.

In [ ]:
eq1 = Π_star.diff(α)
eq2 = Π_star.diff(β)
display(sympy.Equality(eq1, 0))
display(sympy.Equality(eq2, 0))

Which can readily be solved.

In [ ]:
sol = sympy.solve([eq1, eq2], [α, β])
sol

And we find the expressions of $v^\star$, $w^\star$.

In [ ]:
v_star = lambda z : sol[α] * z
v_star(z)

In [ ]:
w_star = lambda z : sol[β] * R
w_star(z)

Remember that we have yet extremized the total potential energy over the space of displacements fields that are of the form $v(z)=\alpha\,z$ and $w(z)=\beta\,R$. We now check that the values of $\alpha$ and $\beta$ found above indeed provide an equilibrium path. To do so, we plug $v^\star$ and $w^\star$ in the residual $\mathrm{D}\Pi$.

In [ ]:
fundamental = {v(x): v_star(x) for x in [0, L, z]}
fundamental.update({w(x): w_star(x) for x in [0, L, z]})
fundamental

In [ ]:
DΠ.subs(fundamental).doit().doit().expand()

Which is indeed 0, since $v(0)=0$.

## Simplified stability analysis of the fundamental branch

To analyse the stability of the fundamental branch, we must find the sign of the following quantity

\begin{equation*}
\Pi[v^\star(q)+\delta v, w^\star(q)+\delta w, q]-\Pi[v^\star(q), w^\star(q), q],
\end{equation*}

for *any* test functions $\delta v$ and $\delta w$ (with $\delta v(0)$).

In the present section, we perform a simplified stability analysis, where we consider buckling modes of the form

\begin{equation*}
\delta v(z) = \delta V\cos(\pi z / L)
\quad\text{and}\quad
\delta w(z) = \delta W\sin(\pi x / L).
\end{equation*}

If, for these specific test functions, the above quantity is negative, then the equilibrium is surely unstable. Conversely, if the quantity is positive, it is not possible to assert the stability of the equilibrium. This approach therefore provides an upper estimate of the critical load.

In [ ]:
δV = sympy.Symbol(r"\delta V")
δW = sympy.Symbol("\delta W")

In [ ]:
perturbation = {v(x): v_star(x) + δV * sympy.sin(sympy.pi * (x / L)) for x in [0, L, z]}
perturbation.update({w(x): w_star(x) + δW * sympy.cos(sympy.pi * x / L) for x in [0, L, z]})
perturbation

In [ ]:
Π1 = Π.subs(perturbation).doit().expand()

In [ ]:
Π1

The perturbed energy is a polynomial function of the two parameters $\delta V$ and $\delta W$. As expected, the linear terms vanish.

In [ ]:
assert Π1.coeff(δV, 1).coeff(δW, 0).factor() == 0
assert Π1.coeff(δV, 0).coeff(δW, 1).factor() == 0

Therefore, to analyse the sign of the perturbed energy in the neighborhood of the fundamental branch, it is sufficient to look at the Hessian as a quadratic form of the variables $\delta V$ and $\delta W$. The coefficients of this quadratic form are extracted below.

In [ ]:
H_vv = Π1.coeff(δV, 2).coeff(δW, 0).factor()
H_vv

In [ ]:
H_ww = Π1.coeff(δV, 0).coeff(δW, 2).factor()
H_ww.expand()

In [ ]:
H_vw = Π1.coeff(δV, 1).coeff(δW, 1).factor() / 2
H_vw

For sufficiently small values of $q$, it can readily be shown that the quadratic form is positive definite (its eigenvalues are positive). For $q=q_\mathrm{cr}$, one of the eigenvalues vanishes, therefore the determinant of the hessian vanishes.

In [ ]:
det_H = sympy.expand(H_vv * H_ww - H_vw**2)
det_H

Nullifying this determinant provides the following estimate of the critical load.

In [ ]:
q_cr = sympy.solve(det_H, q)[0].expand()
q_cr